In [1]:
from haystack.nodes import AnswerParser, PromptModel, PromptNode, PromptTemplate
from haystack.pipelines import Pipeline

from ronin.config import settings

/Users/leonardoschettini/.pyenv/versions/3.11.4/envs/ronin/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt_azure_openai = PromptModel(
    model_name_or_path=settings.azure_openai_chatgpt_deployment, 
    api_key=settings.azure_openai_api_key, 
    model_kwargs={
        "api_version": settings.azure_openai_api_version,
        "azure_base_url": settings.azure_openai_endpoint, 
        "azure_deployment_name": settings.azure_openai_chatgpt_deployment,
    }
)


In [3]:
user_prompt_template = PromptTemplate(
    prompt="Revert, character by character, the message of the user.\n\nUser's message {query}",
    output_parser=AnswerParser(),
)

In [4]:
list(user_prompt_template.fill(query="Hello, how are you?"))

["Revert, character by character, the message of the user.\n\nUser's message Hello, how are you?"]

In [5]:
openai_node = PromptNode(prompt_azure_openai)

In [6]:
user_node = PromptNode(prompt_azure_openai, default_prompt_template=user_prompt_template)

In [7]:
user_node.prompt_model

PromptModel({'_component_config': {'params': {'model_name_or_path': 'gpt-35-turbo-16k', 'api_key': 'd81b3df071cb419e89a61e6b5b54a614', 'model_kwargs': {'api_version': '2023-07-01-preview', 'azure_base_url': 'https://poc-apps-openai-16k.openai.azure.com/', 'azure_deployment_name': 'gpt-35-turbo-16k'}}, 'type': 'PromptModel'}, 'outgoing_edges': 1, 'model_name_or_path': 'gpt-35-turbo-16k', 'max_length': 100, 'api_key': 'd81b3df071cb419e89a61e6b5b54a614', 'use_auth_token': None, 'use_gpu': None, 'devices': None, 'model_kwargs': {'api_version': '2023-07-01-preview', 'azure_base_url': 'https://poc-apps-openai-16k.openai.azure.com/', 'azure_deployment_name': 'gpt-35-turbo-16k'}, 'model_invocation_layer': <haystack.nodes.prompt.invocation_layer.azure_chatgpt.AzureChatGPTInvocationLayer object at 0x28354b450>})

In [9]:
messages = [{"role": "system", "content": "You are a helpful assistant"}, {"role": "user", "content": "Hello, how are you?"}]

user_node(messages)

aaaa PromptTemplate(name=custom-at-query-time, prompt_text=Revert, character by character, the message of the user.{new_line}{new_line}User's message {query}, prompt_params=['query'])
aaaa ([{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hello, how are you?'}],)
aaaa!! Revert, character by character, the message of the user.

User's message {'role': 'system', 'content': 'You are a helpful assistant'}
aaaa!! Revert, character by character, the message of the user.

User's message {'role': 'user', 'content': 'Hello, how are you?'}


[<Answer {'answer': "User's message {'role': 'system', 'content': 'You are a helpful assistant'}", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {'prompt': "Revert, character by character, the message of the user.\n\nUser's message {'role': 'system', 'content': 'You are a helpful assistant'}"}}>,
 <Answer {'answer': "{'role': 'assistant', 'content': '????????? ,uoy era woh olleH'}", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {'prompt': "Revert, character by character, the message of the user.\n\nUser's message {'role': 'user', 'content': 'Hello, how are you?'}"}}>]

In [10]:
openai_node(messages)

aaaa None
aaaa ([{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hello, how are you?'}],)
aaaa!! [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hello, how are you?'}]


["Hello! I'm an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"]

In [11]:
openai_node.run(invocation_context={"messages": messages})

aaaa None
aaaa ()


({'results': [],
  'invocation_context': {'messages': [{'role': 'system',
     'content': 'You are a helpful assistant'},
    {'role': 'user', 'content': 'Hello, how are you?'}],
   'results': [],
   'prompts': []}},
 'output_1')

In [20]:
pipe = Pipeline()
pipe.add_node(component=user_node, name="user_node", inputs=["Query"])

In [21]:
output = pipe.run(query="Why do airplanes leave contrails in the sky?")
output

{'answers': [<Answer {'answer': '?yks eht ni starc nialpertneirpoc evael senialpilA od oD yhW', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {'prompt': "Revert, character by character, the message of the user.\n\nUser's message Why do airplanes leave contrails in the sky?"}}>],
 'invocation_context': {'query': 'Why do airplanes leave contrails in the sky?',
  'answers': [<Answer {'answer': '?yks eht ni starc nialpertneirpoc evael senialpilA od oD yhW', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {'prompt': "Revert, character by character, the message of the user.\n\nUser's message Why do airplanes leave contrails in the sky?"}}>],
  'prompts': ["Revert, character by character, the message of the user.\n\nUser's message Why do airplanes leave contrails in the sky?"]},
 'root_node': 'Query',
 'para